## Red Herring

In [1]:
!pip install transformers torch scikit-learn nltk

  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached regex-2024.11.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached safetensors-0.5.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.me

In [2]:
import nltk
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import classification_report

In [3]:
nltk.download('punkt')

# Red Herring Fallacy examples
red_herring_examples = [
    "We shouldn’t worry about climate change when there are so many people unemployed.",
    "Why talk about school funding? Let’s focus on the real problem: declining family values.",
    "I know I made a mistake at work, but think of all the good things I’ve done.",
    "Sure, I didn’t clean the kitchen, but you never take the trash out!",
    "You're asking about taxes, but what about crime rates?",
    "Let’s not talk about the budget. Have you seen how poorly dressed the mayor is?",
    "We don’t need to fix the potholes. The real issue is people driving too fast.",
    "Yes, we need healthcare reform, but look at how much athletes get paid!",
]

# Healthy counter-arguments (not fallacious)
healthy_arguments = [
    "I think we need to increase school funding because of declining test scores.",
    "While I understand your point, data shows climate change is a growing concern.",
    "It’s true that taxes are high, but healthcare access is crucial.",
    "Yes, unemployment is an issue, but we must also address climate change.",
    "Cleaning the kitchen is important; let’s both take responsibility.",
    "We need to discuss both crime rates and taxation separately.",
]

# Neutral/non-argumentative
non_arguments = [
    "I love watching documentaries on space exploration.",
    "The weather has been nice lately.",
    "I prefer tea over coffee in the mornings.",
    "My dog just learned a new trick.",
]


[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
texts = red_herring_examples + healthy_arguments + non_arguments
labels = [1]*len(red_herring_examples) + [0]*len(healthy_arguments) + [0]*len(non_arguments)


In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encodings = tokenizer(texts, truncation=True, padding=True, return_tensors='pt')


In [6]:
inputs = encodings['input_ids']
masks = encodings['attention_mask']
labels_tensor = torch.tensor(labels)

dataset = TensorDataset(inputs, masks, labels_tensor)
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)


In [7]:
train_loader = DataLoader(train_data, batch_size=2, shuffle=True)
test_loader = DataLoader(test_data, batch_size=2)


In [8]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Training loop (simple)
model.train()
for epoch in range(3):
    for batch in train_loader:
        b_input_ids, b_input_mask, b_labels = batch
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        b_input_ids, b_input_mask, b_labels = batch
        outputs = model(b_input_ids, attention_mask=b_input_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        predictions.extend(preds.tolist())
        true_labels.extend(b_labels.tolist())

print(classification_report(true_labels, predictions))


              precision    recall  f1-score   support

           0       0.25      1.00      0.40         1
           1       0.00      0.00      0.00         3

    accuracy                           0.25         4
   macro avg       0.12      0.50      0.20         4
weighted avg       0.06      0.25      0.10         4



/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [10]:
def predict_red_herring(text):
    # Tokenize the input text
    encoding = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    # Inference mode
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=-1).item()

    return "Red Herring" if prediction == 1 else "Good Argument or Non-Argument"

# 🔍 Example usage
print(predict_red_herring("Why talk about healthcare? What about the cost of stadiums?"))
print(predict_red_herring("I love hiking in the mountains."))


Red Herring
Good Argument or Non-Argument
